In [82]:
import re
from datetime import datetime
import struct


class SaveGameEditor:
    def __init__(self, ext=".dat", root_dir=None, campaign=None):
        self.root_dir = root_dir
        self.campaign = campaign
        self.file = f"{self.root_dir}/{self.campaign}/{self.campaign}{ext}"
        self.read_savegame()
        self.save_backup_savegame()
        
    def save_backup_savegame(self):
        now_str = datetime.now().strftime("%Y%m%d-%H%M%S")
        with open(f"{self.file}-backup-{now_str}", 'wb') as f:
            f.write(self.txt)
    
    def read_savegame(self):
        with open(self.file, 'rb') as f:
            self.txt = f.read()
            
    def save_savegame(self):
        with open(self.file, 'wb') as f:
            f.write(self.txt)
    
    def read_events(self):
        res = re.search(b"Event_City_Campaign_[0-9]{1,2}ID\r", self.txt)
        city_pattern = b"Event_City_Campaign_([a-zA-Z0-9]*)ID"
        self.city_events = [n for n in re.findall(city_pattern, self.txt[:res.span()[1]])]
        self.n_city_events = len(self.city_events)
        res = re.search(b"Event_Road_Campaign_[0-9]{1,2}ID\r", self.txt)
        road_pattern = b"Event_Road_Campaign_([a-zA-Z0-9]*)ID"
        self.road_events = [n for n in re.findall(road_pattern, self.txt[:res.span()[1]])]
        self.n_road_events = len(self.road_events)
    
    @staticmethod
    def prettify_events(events):
        return " ".join([e.decode("utf-8") for e in events])
    
    def print_events_info(self, event=None):
        self.read_events()
        if event == "city" or event is None:
            print(f"{self.n_city_events} City Events:")
            print(f"Current order: {self.prettify_events(self.city_events)}")
            print(f"Sorted: {self.prettify_events(sorted(self.city_events))}")
        if event is None:
            print("")
        if event == "road" or event is None:
            print(f"{self.n_road_events} Road Events:")
            print(f"Current order: {self.prettify_events(self.road_events)}")
            print(f"Sorted: {self.prettify_events(sorted(self.road_events))}")

    @staticmethod
    def get_events_span(events_txt, event="city"):
        event_capital = b"City" if event == "city" else b"Road"
        return [
            {"event_number": int(re.search(b'_([0-9]*)ID', e.group()).group(1)),
             "event_span": e.span()} 
            for e in list(re.finditer(
                b"(\x17|\x18)Event_" + event_capital + b"_Campaign_[a-zA-Z0-9]*ID",
                events_txt
            ))
        ]
    
    def replace_events(self, event="city", new_events=None, verbose=True):
        if not new_events:
            print("You didn't specify new events to replace the existing events with!")
            return
        if event == "city":
            if len(new_events) != self.n_city_events:
                msg = "You should supply the same number of events as there are currently in the savegame!\n" + \
                    f"There are {self.n_city_events} city events in the deck. But you supplied {len(new_events)}."
                if self.n_city_events < len(new_events):
                    msg += f"\nPlease start the game and unlock {len(new_events) - self.n_city_events}" + \
                        f" extra city event(s) through the merchant, then try again."
                else:
                    msg += f"\nI replaced all the city events with event 33, which can easily be triggered and" + \
                        f" then discarded.\nSo, please start the game and trigger {self.n_city_events - len(new_events)}" + \
                        f" city event(s), each time choosing the bottom option.\nThen try again."
                    new_events = [33]*self.n_city_events
                    self.replace_events(event=event, new_events=new_events)
                print(msg)
                return
        if event == "road":
            if len(new_events) != self.n_road_events:
                msg = "You should supply the same number of events as there are currently in the savegame!\n" + \
                    f"There are {self.n_road_events} road events in the deck. But you supplied {len(new_events)}."
                if self.n_road_events < len(new_events):
                    msg += f"\nPlease start the game and unlock {len(new_events) - self.n_road_events}" + \
                        f" extra road event(s), then try again."
                else:
                    msg += f"\nI replaced all the road events with event 16, which can easily be triggered and" + \
                        f" then discarded.\nSo, please start the game and trigger {self.n_road_events - len(new_events)}" + \
                        f" road event(s), each time choosing the bottom option.\nThen try again."
                    new_events = [16]*self.n_road_events
                    self.replace_events(event=event, new_events=new_events, verbose=False)
                print(msg)
                return
        event_capital = b"City" if event == "city" else b"Road"
        
        # Find the location of the event data within the full text
        events_start_index = list(
            re.finditer(b"Event_" + event_capital + b"_Campaign_[a-zA-Z0-9]*ID", self.txt)
        )[0].start() - 1
        events_end_index = list(
            re.finditer(b"Event_" + event_capital + b"_Campaign_[a-zA-Z0-9]*ID\r", self.txt)
        )[0].end()
        events_txt = self.txt[events_start_index:events_end_index]
        
        # Find all the individual events within the text, return their exact location in the string
        # and return the text as a list of substrings that we can then alter
        event_infos = self.get_events_span(events_txt, event=event)  
        events_txt_list = []
        for i, event_info in enumerate(event_infos):
            next_str = events_txt[event_info["event_span"][0]:event_info["event_span"][1]]
            events_txt_list.append(next_str)
            if i < len(event_infos)-1:
                next_str = events_txt[event_info["event_span"][1]:event_infos[i+1]["event_span"][0]]
                events_txt_list.append(next_str)
            else:
                next_str = events_txt[event_info["event_span"][1]:]
                events_txt_list.append(next_str)
        # Alter all the events into the correct events
        for i, new_event in enumerate(new_events):
            if new_event >= 10:
                event_str = b"\x18"
            else:
                event_str = b"\x17"
            event_str = event_str + b"Event_" + event_capital + b"_Campaign_" + \
                bytes(str(new_event), "utf-8") + b"ID"
            events_txt_list[i*2] = event_str
            
        # Recreate the event text and the full text
        new_events_txt = b''.join(events_txt_list)
        new_txt = self.txt[:events_start_index] + new_events_txt + self.txt[events_end_index:]
        self.txt = new_txt
        if verbose:
            self.print_events_info(event=event)
        
    def update_gold_and_exp(self, char_name="Sol Goodman", gold=None, exp=None):
        char_info_span = re.search(bytes(char_name, "utf-8") + b'(?s:.)*?ID(.*?)', self.txt).span(1)
        gold_span = (char_info_span[0], char_info_span[0]+4)
        exp_span = (char_info_span[0]+4, char_info_span[0]+8)
        level_span = (char_info_span[0]+8, char_info_span[0]+12)
        current_gold = struct.unpack("<I", self.txt[gold_span[0]:gold_span[1]])[0]
        current_exp = struct.unpack("<I", self.txt[exp_span[0]:exp_span[1]])[0]
        current_level = struct.unpack("<I", self.txt[level_span[0]:level_span[1]])[0]
        if gold:
            new_gold_str = struct.pack('<I', gold)
            self.txt = self.txt[0:gold_span[0]] + new_gold_str + self.txt[gold_span[1]:]
            new_gold = struct.unpack("<I", self.txt[gold_span[0]:gold_span[1]])[0]
            print(f"Updated {char_name}'s gold amount from {current_gold} to {new_gold}.")
        else:
            print(f"{char_name} currently has {current_gold} gold.")
        if exp:
            new_exp_str = struct.pack('<I', exp)
            self.txt = self.txt[0:exp_span[0]] + new_exp_str + self.txt[exp_span[1]:]
            new_exp = struct.unpack("<I", self.txt[exp_span[0]:exp_span[1]])[0]
            print(f"Updated {char_name}'s experience from {current_exp} (level {current_level}) to {new_exp}.")
        else:
            print(f"{char_name} currently is level {current_level} with {current_exp} experience.")

In [83]:
root_dir = "/Users/EQ81TW/Library/Application Support/unity." + \
            "FlamingFowlStudios.Gloomhaven/GloomSaves/Campaign"
campaign = "Campaign_[MOD]TabletopToDigital[MOD]_The_Starbase_Raptors_80287552"
editor = SaveGameEditor(root_dir=root_dir, campaign=campaign)
editor.print_events_info()

19 City Events:
Current order: 18 3 57 41 26 71 37 27 29 21 10 31 25 2 13 1 34 24 30
Sorted: 1 10 13 18 2 21 24 25 26 27 29 3 30 31 34 37 41 57 71

24 Road Events:
Current order: 3 33 40 5 31 42 24 30 57 47 2 19 11 7 9 41 6 4 44 12 25 14 15 1
Sorted: 1 11 12 14 15 19 2 24 25 3 30 31 33 4 40 41 42 44 47 5 57 6 7 9


In [84]:
editor.replace_events(event="city", new_events=[
    18, 3, 57, 41, 26, 71, 37, 27, 29, 21, 10, 31, 25, 2, 13, 1, 34, 24, 30
])

19 City Events:
Current order: 18 3 57 41 26 71 37 27 29 21 10 31 25 2 13 1 34 24 30
Sorted: 1 10 13 18 2 21 24 25 26 27 29 3 30 31 34 37 41 57 71


In [85]:
editor.replace_events(event="road", new_events=[
    3, 33, 40, 5, 31, 42, 24, 30, 57, 47, 2, 19, 11, 7, 9, 41, 6, 4, 44, 12, 25, 14, 15, 1
])

24 Road Events:
Current order: 3 33 40 5 31 42 24 30 57 47 2 19 11 7 9 41 6 4 44 12 25 14 15 1
Sorted: 1 11 12 14 15 19 2 24 25 3 30 31 33 4 40 41 42 44 47 5 57 6 7 9


In [86]:
editor.update_gold_and_exp("Sol Goodman", gold=54, exp=289)
editor.update_gold_and_exp("Nine Lives Lilly", gold=None, exp=None)
editor.update_gold_and_exp("Doc Morbid", gold=None, exp=None)
editor.update_gold_and_exp("Emesh", gold=None, exp=None)

Updated Sol Goodman's gold amount from 54 to 54.
Updated Sol Goodman's experience from 289 (level 5) to 289.
Nine Lives Lilly currently has 27 gold.
Nine Lives Lilly currently is level 6 with 308 experience.
Doc Morbid currently has 2 gold.
Doc Morbid currently is level 6 with 276 experience.
Emesh currently has 42 gold.
Emesh currently is level 4 with 174 experience.


In [73]:
editor.save_savegame()